In [ ]:
import os
import sys
from dotenv import load_dotenv
import json
import pandas as pd
from weaviate_rag.rag_system import GraphRAGSystem
import ollama
from ragas import evaluate
from ragas.metrics import Faithfulness, AnswerRelevancy, ContextRecall
import re
import time
from ragas import EvaluationDataset
from together import Together

# Initialize Together client
load_dotenv()

MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
TOGETHER_API_KEY = os.environ["TOGETHER_API_KEY"]

client = Together(api_key=TOGETHER_API_KEY)

In [ ]:
# Load your JSON data
with open('/Users/alexlecu/PycharmProjects/LLMKGraph/backend/evaluation/data/grok_evaluation_datasets/2_Hop_OpenEnded_Questions.json', 'r') as f:
    data = json.load(f)

In [ ]:
questions = [entry['question'] for entry in data]
ground_truths = [entry['answer'] for entry in data]

In [ ]:
model_config = {
    'provider': 'together',
    'model_name': MODEL_NAME,
    'api_key': TOGETHER_API_KEY
}

def retrieve(user_input):
    analyzer = GraphRAGSystem(user_input, model_config)
    context = analyzer.analyze()

    return [context]

In [ ]:
contexts = [retrieve(q) for q in questions]

In [ ]:
def generate_answer(question, context, model=MODEL_NAME):
    user_prompt = f"""
    You are a trusted medical research assistant specializing in age-related macular degeneration (AMD). Your task is to provide concise, accurate, and factual answers about AMD research, limited to one sentence, based on the following additional relevant data:
    
    {context}
    
    Please adhere to these guidelines when formulating your response:

    1. Be Concise:
    Provide a direct answer in one sentence, focusing on the most essential information without unnecessary elaboration.
    2. Express Uncertainty Transparently:
    If the available information is insufficient to answer confidently, acknowledge this and specify what additional data or details would be needed to provide a more complete response.
    3. Maintain Accuracy and Integrity:
    Base your answer solely on verified data and the provided context. Do not fabricate any information or references.
    4. Communicate Professionally:
    Present your response in a clear, well-organized, and professional manner, ensuring complex information is accessible and easy to understand.

    Question:
    {question}

    Begin your response below.
    """
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a medical knowledge assistant."},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1,
            top_p=0.9,
            max_tokens=200
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error with Together API: {e}")
        time.sleep(1)  # Rate limiting
        return None

In [ ]:
answers = [generate_answer(q, c) for q, c in zip(questions, contexts)]

In [ ]:
evaluation_data = pd.DataFrame({
    'question': questions,
    'ground_truth': ground_truths,
    'context': contexts,
    'generated_answer': answers
})

evaluation_data.to_csv('ragas_2hop_llama8b.csv', index=False)